# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI


In [27]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [28]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")
select_relevant_links("https://www.vrt.be/vrtnws/nl/")

{'links': [{'type': 'ethics page',
   'url': 'https://www.vrt.be/nl/over-de-vrt/beleid/beroepsethiek/deontologische-code-en-redactiestatuut/'},
  {'type': 'careers page', 'url': 'https://www.vrt.be/nl/jobs/'},
  {'type': 'accessibility statement',
   'url': 'https://vrt.be/toegankelijkheidsverklaring'},
  {'type': 'contact page', 'url': 'https://www.vrt.be/nl/contact/'},
  {'type': 'ai policy',
   'url': 'https://www.vrt.be/vrtnws/nl/algemeen/ai-beleid/'},
  {'type': 'policy',
   'url': 'https://www.vrt.be/nl/info/responsible-disclosure-policy/'},
  {'type': 'social media', 'url': 'https://twitter.com/vrtnws/'},
  {'type': 'social media', 'url': 'https://www.tiktok.com/@vrt.nws/'},
  {'type': 'social media', 'url': 'https://www.facebook.com/vrtnws/'},
  {'type': 'social media', 'url': 'https://bsky.app/profile/vrtnws.be'},
  {'type': 'social media', 'url': 'https://www.instagram.com/vrtnws/'},
  {'type': 'social media',
   'url': 'https://www.youtube.com/channel/UC59gT3bFTFNSqafRcluDIs

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio / project',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio / project', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'curriculum / services',
   'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'proficiencies / services',
   'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 18 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'endpoints product page',
   'url': 'https://endpoints.huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 15 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-5
Updated
1 day ago
•
1.55k
•
757
openbmb/MiniCPM-o-4_5
Updated
about 4 hours ago
•
40.4k
•
996
openbmb/MiniCPM-SALA
Updated
1 day ago
•
117
•
482
Qwen/Qwen3-Coder-Next
Updated
9 days ago
•
185k
•
798
moonshotai/Kimi-K2.5
Updated
8 days ago
•
648k
•
2.08k
Browse 2M+ models
Spaces
Running
on
A100
Featured
329
ACE-Step v1.5
🎵
329
Music Generation Foundation Model v1.5
Running
on
Zero
Featured
1.39k
Qwen3-TTS Demo
🎙
1.39k
Generate speech from text with voice design, cloning, or speakers
Running
594
Demo Playground
⚡
594
Free

In [24]:
#brochure_system_prompt = """
#You are an assistant that analyzes the contents of several relevant pages from a company website
#and creates a short brochure about the company for prospective customers, investors and recruits.
#Respond in markdown without code blocks.
#Include details of company culture, customers and careers/jobs if you have the information.
#"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-5\nUpdated\n1 day ago\n•\n1.55k\n•\n757\nopenbmb/MiniCPM-o-4_5\nUpdated\nabout 4 hours ago\n•\n40.4k\n•\n996\nopenbmb/MiniCPM-SALA\nUpdated\n1 day ago\n•\n117\n•\n482\nQwen/Qwen3-Coder-Next\nUpdated\n9 days ago\n•\n185k\n•\n798\nmoonshotai/Kimi-K2.5\nUpdated\n8 days ago\n•\n648k\n•\n2.08k\nBrowse 2M+ models\nSpaces\nRunning\non\nA100\nFeatured\n330\nACE-Step v1.5\n🎵\n330\nMusic 

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the premier collaboration platform for the machine learning (ML) community and a driving force in building the future of artificial intelligence. It serves as the central hub where machine learning engineers, researchers, and enthusiasts come together to create, share, and explore over 2 million models, 500k+ datasets, and more than 1 million AI applications.

The platform supports a wide array of ML modalities including text, image, video, audio, and even 3D data, making it versatile and comprehensive for diverse AI projects. Hugging Face fosters an open and ethical AI future by empowering its community to learn, collaborate, and innovate together.

---

## Key Features & Offerings

- **Models:** Access to an extensive library of over 2 million open-source machine learning models, including state-of-the-art models like GLM-5, MiniCPM, Qwen coder models, and many more.
- **Datasets:** Discover and contribute over 500,000 datasets spanning various domains to fuel your AI experiments.
- **Spaces:** Explore and deploy AI-powered applications and demos with ease. These community-driven spaces showcase music generation, speech synthesis, image generation, and more.
- **Community Hub:** A thriving ecosystem where developers share projects, build portfolios, and collaborate on cutting-edge AI technologies.
- **Enterprise Solutions:** Scalable paid compute and enterprise-grade solutions offering advanced security, access controls, and dedicated support for teams and businesses to accelerate AI development.
- **Open Source Stack:** A robust open-source software toolkit designed to enable faster prototyping and deployment of AI models.

---

## Our Community & Customers

Hugging Face is trusted by a vibrant, fast-growing community of ML engineers, data scientists, researchers, and enterprise clients who value openness and collaboration. It is used by academic institutions, startups, and leading global tech companies to:

- Host and collaborate on public and private ML models and datasets.
- Develop novel AI applications across disciplines.
- Build transparent and ethical AI systems by leveraging community knowledge.
- Accelerate research and product development with cutting-edge tools and shared resources.

---

## Company Culture and Values

- **Collaboration:** Hugging Face thrives on collective intelligence. The platform encourages sharing and co-creation to push the boundaries of AI.
- **Openness:** The company strongly supports open source AI development, believing transparency fuels innovation and trust.
- **Ethical AI:** Committed to building responsible AI technology that benefits society.
- **Inclusivity:** Hugging Face welcomes a diverse range of contributors, from seasoned researchers to newcomers in AI.
- **Learning & Growth:** Members are encouraged to build their machine learning profiles, learn continuously, and contribute to the evolution of AI.

---

## Careers

Join Hugging Face and be part of a community that is shaping the future of AI. The company offers exciting career opportunities for:

- Machine learning engineers and researchers
- Software developers and architects specializing in AI
- Community managers and developer advocates
- Product managers with AI experience
- Enterprise solution engineers

Employees work in a dynamic, innovative environment focused on open-source projects and impactful AI advancements. Hugging Face values creativity, autonomy, and continuous learning.

---

## Summary

Hugging Face is the home of machine learning where anyone can create, discover, and collaborate on AI projects. Whether you are an individual contributor, part of a research team, or an enterprise looking to accelerate AI adoption, Hugging Face offers the most powerful tools, datasets, and community support for building the AI of tomorrow.

**Explore, experiment, and evolve with Hugging Face — The AI community building the future.**

---

### Visit us: [huggingface.co](https://huggingface.co)

### Contact & Careers: [huggingface.co/careers](https://huggingface.co/careers)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [29]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


# Hugging Face Brochure

---

## Who We Are

Hugging Face is a thriving AI community and collaboration platform dedicated to building the future of machine learning (ML). We empower ML engineers, scientists, researchers, and enthusiasts worldwide to share, explore, and experiment with open-source ML models, datasets, and applications. Our platform acts as the home of machine learning—where innovation is fueled by connection and collaboration.

---

## Our Platform

- **Open Collaboration:** Host and collaborate on unlimited public machine learning models, datasets, and AI applications.
- **Wide Modalities:** Explore and build using diverse modalities including text, image, video, audio, and even 3D.
- **Extensive Resources:** Access over 2 million models, 500,000+ datasets, and 1 million+ AI applications.
- **Community-Driven:** A growing ecosystem of users who continuously contribute to trending, cutting-edge ML projects.
- **Portfolio Building:** Share your work with the world through profiles and portfolios designed to showcase your AI expertise.
- **Enterprise & Compute Solutions:** Accelerate ML projects with paid compute resources and enterprise-grade solutions featuring robust security and access controls.

---

## Our Community & Customers

Hugging Face serves a diverse community:
- **Individual ML practitioners:** Developers, data scientists, and researchers leveraging our open source stack.
- **Teams & Enterprises:** Companies looking for scalable, secure platforms for AI development.
- **Academic & Research Organizations:** Collaborating on novel models and datasets.
- **AI Enthusiasts and Learners:** Access to a vast repository of resources to learn and grow in the AI field.

Our platform supports both public contributions and private enterprise projects, fostering innovation while ensuring security and compliance.

---

## Company Culture

- **Open & Ethical AI:** We are committed to building an open and ethical AI future together.
- **Community Empowerment:** We empower the next generation of ML engineers and scientists to learn and grow through collaboration.
- **Innovation-Driven:** Our fast-growing community thrives on sharing and pushing the boundaries of machine learning.
- **Transparency and Sharing:** Embracing open source principles that encourage knowledge exchange without barriers.

---

## Careers at Hugging Face

Join us if you are passionate about:
- Building scalable and user-friendly AI platforms
- Contributing to open source machine learning tools
- Collaborating with a global community of AI innovators
- Advancing the frontiers of ethical AI and responsible technology

We are looking for talented engineers, data scientists, community managers, and enterprise solution experts who want to shape the future of machine learning.

---

## Why Choose Hugging Face?

- **Largest open ML community:** Access to an unparalleled breadth of models and datasets.
- **Collaborative ecosystem:** Co-create with peers and leading experts worldwide.
- **Advanced tools and infrastructure:** From individual projects to enterprise deployments, we have you covered.
- **Ethical AI commitment:** Join a movement focused on transparency and positive impact.

---

## Get Started

Explore our platform:
- Browse AI models: 2M+ models available
- Dive into datasets: 500k+ datasets
- Try AI apps and demos in Hugging Face Spaces
- Sign up to create your portfolio and start collaborating

Become part of the AI community that’s building the future.  
Visit us at [huggingface.co](https://huggingface.co)

---

*Hugging Face – The AI community building the future.*

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Welcome to Hugging Face: Where AI Gives You a Hug and a High-Five! 🤗🤖

---

## Who Are We?

Hugging Face isn’t just a company; it’s the bustling **AI community building the future**—one model, one dataset, and one witty bot at a time. Picture a global playground where machine learning geeks, coding wizards, and AI dreamers unite to create, collaborate, and celebrate the magic of open-source AI.

---

## What’s the Big Deal?

- **2 MILLION+ AI Models:** From text to images, video to 3D, audio to everything under the silicon sun — we host more open-source models than a cat has lives.
  
- **500,000+ Datasets:** Math problems, 3D benchmarks, NLP goldmines—you name it, we’ve got the data that fuels tomorrow’s AI innovations.
  
- **1 MILLION+ AI Applications:** Try out apps running on turbocharged A100 GPUs or humble CPUs. They do everything from *speech synthesis* to *high-quality image generation*. Hello, AI DJ and Picasso bots!

- **Spaces:** Your playground for hosting and sharing AI demos. Want to build your portfolio or just wow your friends? Spaces let you run your models in the cloud without breaking a sweat.
  
---

## What Makes Us Tick? 

- **Community First:** Whether you’ve just taught your grandma to say “Hello, World!” or you’re building the next AI that writes sonnets or solves quantum physics, Hugging Face is your AI home. Share your models and datasets, collaborate freely, and ride the wave of innovation.

- **Open-Source AF:** We believe the best AI is open AI. That’s why our entire stack is designed so you can move fast and break things responsibly.

- **Enterprise Ready:** For companies that want to build AI with security, access controls, and dedicated support, our enterprise solutions offer a first-class AI ride without the turbulence.

---

## For Customers & Users

Why settle for boring AI when you can explore models like:

- **Qwen3-Coder-Next:** Your AI coding buddy that’s smarter than your average programmer (and less likely to steal your lunch).
- **ACE-Step v1.5:** Music generation that hits all the right notes without an off-key tweet.
- **Demo Playground:** Where you can play with multiple AI models for free — because why should trial-and-error *not* be fun?

Whatever your AI cravings—text, audio, images, or quirky new modalities—Hugging Face has the recipe.

---

## For Investors

Invest in the **heartbeat of AI innovation**. Hugging Face stands tall as the collaboration platform *powering* the machine learning community. With millions of users, models, datasets, and apps, we’re not just riding the AI wave—we’re setting the course for the future. Join us in nurturing an open, ethical AI ecosystem that benefits engineers, businesses, and end users alike.

---

## For Future Hugging Faces (Careers)

Love ML? Like to collaborate? Fancy yourself as part of a vibrant bunch reinventing AI daily?

- Work with a **passionate, open community** that thrives on sharing knowledge and smashing silos.
- Build tools and platforms that make AI accessible to everyone—from hobbyists to Fortune 500s.
- Grow your career in a space where your contributions *literally* help shape the AI frontier.

Find your place where the future is made... and given a virtual bear hug.

---

## Quick Facts

- Colors to brighten your day: **#FFD21E (Hug Yellow), #FF9D00 (Orange You Glad?), and #6B7280 (Sensible Gray)**
- Logos available in every file format you can throw at your design team — SVG, PNG, AI. Yes, we’re that ready.
- We believe AI is for everyone. Ethical, open, and fun.

---

## How to Join the Party

Whether you want to browse **2 million+ models**, explore **500k+ datasets**, or launch your own **AI Space**, come sign up and join the most vibrant ML community on Earth.

[Sign Up & Hug an Algorithm Today!](https://huggingface.co)

---

*Hugging Face – More than code, it’s a community.*  
*Where your AI ambitions get both the hug and the high-five.* 🤗✋

---

Ready to hug the future? Because we’ve got plenty of AI arms to spare!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>